In [1]:
%load_ext bigdata

In [2]:
%pig_start

In [3]:
%timeout 300

In [7]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
rm: `output/*': No such file or directory
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [8]:
!hadoop fs -put *.tsv .
!hadoop fs -ls

put: `data.tsv': File exists
Found 2 items
-rw-r--r--   1 root supergroup        230 2020-01-18 20:56 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-01-18 22:32 truck_event_text_partition.csv


In [9]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [12]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 5
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:CHARARRAY, f3:CHARARRAY);
columna = FOREACH u GENERATE FLATTEN($1);
dato1 = FOREACH columna GENERATE FLATTEN(TOKENIZE($0)) AS letra;
agrupar = GROUP dato1 BY letra;
contar = FOREACH agrupar GENERATE group, COUNT($1);
limite = LIMIT contar 7;
DUMP limite;


-- escribe el archivo de salida
STORE limite INTO 'output';

-- copia los archivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [13]:
!pig -execute 'run script.pig'

2020-01-18 22:55:33,007 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-18 22:55:35,896 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-18 22:55:36,226 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-18 22:55:36,231 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-18 22:55:36,249 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2020-01-18 22:55:36,269 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.

In [14]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-18 22:57 output/_SUCCESS
-rw-r--r--   1 root supergroup         34 2020-01-18 22:57 output/part-r-00000


In [15]:
!hadoop fs -cat output/part-r-00000

a	22
b	12
c	17
d	13
e	15
f	25
g	9


In [16]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [17]:
!rm -rf input
!rm -rf output